# Build prediction for AICROWD

In [ ]:
import numpy as np
from utils_predictions_manipulation import*
from utils_nans_manipulation import*
from cross_validation import*
from utils_data_loading import*
from utils_features_manipulation import*
from time import time
from logistic_regression import*
#from standardization import*
from least_squares import*

# Load data

In [ ]:
traindata,_ = load_data('Data/train.csv')

In [ ]:
testdata,_ = load_data('Data/test.csv')

In [ ]:
X_train, Y_train = structure_data(traindata)

In [ ]:
X_test,_ = structure_data(testdata)

# Data preparation

In [ ]:
X_nans = replace_bad_data_with_nans(X_train, -999)
X_nans_test = replace_bad_data_with_nans(X_test, -999)

In [ ]:
X_clean,del_indexes = replace_nans_with_median(X_nans, 0.5)

In [ ]:
X_test_del = delete_nans_indexes(X_nans_test, del_indexes)

In [ ]:
X_test_clean = replace_test_nans_with_median(X_test_del, X_clean)

# Logit fine tuned

## Prepping data

In [ ]:
X_train_poly = build_poly_multi(X_clean, 4)
X_train_final,_,_=standardize(X_train_poly)

In [ ]:
X_test_poly = build_poly_multi(X_test_clean, 4)
X_test_final,_,_=standardize(X_test_poly)

In [ ]:
X_train_final.shape

In [ ]:
X_test_final.shape

## Building prediction

In [ ]:
initial_w = np.ones((X_train_final.shape[1]))*(-0.01)

ws,loss = logistic_regression(Y_train, X_train_final, initial_w, max_iters=10000, gamma=0.45)

Y_test = sigmoid(X_test_final.dot(ws))

In [ ]:
Y_pred_logit = probability_to_prediction(Y_test)

## Outputting

In [ ]:
ids = testdata[['Id']]



Y_final = np.c_[np.array(ids, dtype=np.int64), Y_pred]

np.savetxt("submission.csv", Y_final, delimiter=',', header="Id,Prediction", comments="", fmt='%d')

In [ ]:
Y_final

# Manu's thing

In [ ]:
X_test_clean

In [ ]:
X_s = standardize_data(X_clean)
X_s_test = standardize_data(X_test_clean)

#Obtained from genetic algorithm
index = (1, 3, 5, 7, 13, 16, 18, 20, 23)
degree = [(1, 0.25, 0.3333333333333333, 2), (1, 0.03333333333333333, 0.1, 0.2, 0.3333333333333333, 0.5, 2), (1, 0.1, 0.3333333333333333, 20), (1, 0.03333333333333333, 0.05, 0.2, 0.3333333333333333, 30), (1, 0.05, 10), (1, 0.2, 0.25, 0.3333333333333333, 3, 4, 5), (1, 3, 10, 20), (1, 0.1, 0.3333333333333333, 4), (1, 0.03333333333333333, 0.2), (1, 0.1, 2, 5), (1, 0.05, 0.1), (1, 0.2, 0.3333333333333333, 0.5, 2, 4, 10), (1, 0.1, 0.25, 0.3333333333333333, 5, 30), (1, 3, 30), (1, 0.1, 5, 10), (1, 0.25, 3, 30), (1, 0.1, 10, 30), (1, 0.2, 1, 3), (1, 0.05, 0.5, 30), (1, 3, 4), (1, 0.3333333333333333, 3, 5), (1, 2, 10), (1, 0.25, 0.3333333333333333), (1, 0.05, 0.2, 30)]

X_poly = build_poly_index(X_s, index, degree)
X_poly_test = build_poly_index(X_s_test, index, degree)

In [ ]:
meanacc_dtest_ls = []
meanacc_dtrain_ls = []

dtmp_tr,dtmp_te=cross_validation(Y_train,X_poly,k_fold=4,seed=1, function_name='least_squares')
meanacc_dtest_ls.append(dtmp_te)
meanacc_dtrain_ls.append(dtmp_tr)

In [ ]:
meanacc_dtest_ls

In [ ]:
ws,loss = least_squares(Y_train, X_poly)

Y_test = X_poly_test.dot(ws)

Y_pred = probability_to_prediction(Y_test)

In [ ]:
ids = testdata[['Id']]



Y_final = np.c_[np.array(ids, dtype=np.int64), Y_pred]

np.savetxt("submission.csv", Y_final, delimiter=',', header="Id,Prediction", comments="", fmt='%d')

In [ ]:
Y_final